# Read in Ordered and Unordered Transfer Values

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
unordered_pivot = pd.read_csv('unordered_pivot.csv',index_col=0)
transfers = pd.read_csv('unclipped_transfer_table.csv')

# Compute Support with Order

In [3]:
unordered = unordered_pivot.reset_index().iloc[:,:-1].melt(id_vars=['index'])
unordered
unordered = unordered.set_index('index')
unordered.columns = ['Gene Family','Support']
unordered = unordered[unordered['Support'] > 0]
unordered['Forward'] = 0
unordered['Backward'] = 0

In [4]:
for i, row in tqdm(enumerate(unordered.iterrows()),total=len(unordered)):
    first = row[0].split('+')[0]
    second = row[0].split('+')[1]
    gf = row[1]['Gene Family']
    forward = transfers[(transfers['Donor'] == first) &\
                        (transfers['Recipient'] == second) &\
                        (transfers['Gene Family'] == gf)]
    if len(forward):
        forward = forward['Support'].item()
    else:
        forward = 0
    unordered.iloc[i,2] = forward
    backward = transfers[(transfers['Recipient'] == first) &\
                         (transfers['Donor'] == second) &\
                         (transfers['Gene Family'] == gf)]
    if len(backward):
        backward = backward['Support'].item()
    else:
        backward = 0
    unordered.iloc[i,3] = backward

  0%|          | 0/1170 [00:00<?, ?it/s]

In [5]:
unordered['Forward Pct'] = unordered['Forward'] / (unordered['Forward'] + unordered['Backward']).fillna(0)

In [6]:
bidirectional = unordered[(unordered['Forward Pct'] >= 0.4) &\
                          (1 - unordered['Forward Pct'] >= 0.4) &\
                          (unordered['Forward'] >= 100) &\
                          (unordered['Backward'] >= 100)]
bidirectional = bidirectional.sort_values(by=['Support','Forward Pct'],ascending=False)

In [7]:
bidirectional.head()

,Gene Family,Support,Forward,Backward,Forward Pct
index,,,,,
n2+n35,ORF1ab,1000.0,537,463,0.537
Rp3+RpS11,M,1000.0,533,467,0.533
Rs4081+YN2018D,S,1000.0,511,489,0.511
Anlong-103+YN2013,S,1000.0,503,497,0.503
HuB2013+Rm1,ORF1ab,1000.0,480,520,0.480


In [8]:
print(f"Number of Bidirectional transfers: {len(bidirectional)}")

Number of Bidirectional transfers: 48


In [9]:
bidirectional.to_csv('bidirectional_transfers.csv')